In [7]:
!pip install transformers torch accelerate torchtune huggingface_hub

Hugging Face Login


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from huggingface_hub import login

login()

KeyboardInterrupt: 

Model information

In [1]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")

NameError: name 'AutoTokenizer' is not defined

Test Prompts

In [ ]:
# Input prompt
input_text = "Please generate a text based riddle."
inputs = tokenizer(input_text, return_tensors="pt")

# Generate text
outputs = model.generate(
    inputs["input_ids"].to("cuda"),  # Send input to GPU if available
    max_length=1000,  # Maximum length of the output
    num_return_sequences=1,  # Number of responses
    temperature=0.8,  # Adjust creativity level
    top_p=0.9,  # Nucleus sampling
)

# Decode and print the output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
output = None # insert prompt to LLM here

Parser class

In [3]:
class Parser:
  def __init__(self):
    self.lives = 3
    self.hint = None
    self.question = None
    self.answer = None
    self.level = 0

  def ask(self):
    # promtpts the LLM to output a new riddle in the form of a json
    #
    #
    if (self.question != None):
      return self.question
    else:
      # Input prompt
      input_text = "Please generate a text based riddle."
      inputs = tokenizer(input_text, return_tensors="pt")

      # Generate text
      outputs = model.generate(
          inputs["input_ids"].to("cuda"),  # Send input to GPU if available
          max_length=1000,  # Maximum length of the output
          num_return_sequences=1,  # Number of responses
          temperature=0.8,  # Adjust creativity level
          top_p=0.9,  # Nucleus sampling
      )

      # Decode and print the output
      generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
      output = None # insert prompt to LLM here

      self.question = generated_text #PLACEHOLDER. JUST RETURNS MODEL OUTPUT CURRENTLY
      self.hint = generated_text #PLACEHOLDER. JUST RETURNS MODEL OUTPUT CURRENTLY
      self.answer = generated_text #PLACEHOLDER. JUST RETURNS MODEL OUTPUT CURRENTLY

      return self.question

  def game(self):
    # Game - Provides the game loop and main functionality of the game.
    # The game ends when
    #
    playing = True
    while(playing & (self.lives >= 0 or self.level > 3)):
      # Prompts ask to generate the question and parse it
      print(self.ask())

      #
      response = input("")
      response = response.lower()
      responseList = response.split(" ")
      if(responseList[0] == "q" or responseList[0] == "quit"):
        playing = False # User
      if(responseList[0] == "give" and responseList[1] == "up"):
        print("You decide to move on to the next question")
        self.life -= 1
        self.hint = None
        self.question = None
        self.answer = None
      if(responseList[0] == "help" or responseList[0] == "h"):
        print(self.hint)

      if(False): #Placeholder for succeeding the riddle. We'll add an evaluation method later.
        # Tells player they lost, decrements lives, and resets hint, question and answer variables
        print("Congrats that was the correct response")
        self.level += 1
        self.hint = None
        self.question = None
        self.answer = None
      elif (False): #Placeholder for succeeding the riddle. We'll add an evaluation method later
        print("Incorrect response. Please try again")
        self.life -= 1


    # The game has ended.
    if(self.lives >= 0)
      print("You lose and explode into a million pieces on the spot.")
    if(self.level > 3)
      print("Congratulations! You're free from the sphinx's game and did not explode.")
    return


In [ ]:
pharohs_game = Parser()
pharohs_game.game()